In [29]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
def split_color(img, value=50):
    img[img<value]=0
    img[img>=value]=200
    return img
    
def compute_image_iou(pred_file,label_file):
    label_image = np.array(Image.open(label_file).resize((512,512)))
    label_image = np.expand_dims(label_image,-1)
    label_image = np.concatenate([label_image,label_image,label_image],axis=2)
    pred_image = np.array(Image.open(pred_file).resize((512,512)))
    label_image = split_color(label_image,0.5)
    pred_image = split_color(pred_image)
    plt.rcParams['figure.dpi'] = 300
    plt.subplot(1,2,1)
    plt.imshow(pred_image)
    plt.subplot(1,2,2)
    plt.imshow(label_image)
    background_color=np.array([0, 0, 0])
    #print('[I] Shape:',label_image.shape, pred_image.shape)
    #print(label_file, pred_image)
    bg_image = np.all(label_image == background_color, axis=2)
    #plt.imshow(bg_image)
    label = np.invert(bg_image)

    bg_image = np.all(pred_image == background_color, axis=2)
    pred = np.invert(bg_image)
    
    label = label.reshape(-1,1).astype(float)
    pred = pred.reshape(-1,1).astype(float)
    inter = np.sum(label * pred)
    union = max(np.sum(label + pred) - inter, 1)
    iou = inter*1./union
    return iou
def batch_iou(pred_dir,label_dir):
    iou = 0
    count=0
    for i in os.listdir(pred_dir):
        if i.split('.')[-1]!='png':
            continue
        name=i.split('_')[0].split('.')[0].split('-')[-1]
        pred_file = os.path.join(pred_dir,'testing-'+name+'.png')
        label_file = os.path.join(label_dir,'RGB-PanSharpen_AOI_2_Vegas_'+name+'_mask.png')
        if not os.path.exists(label_file):
            continue
        # print(label_file)
        iou+=compute_image_iou(pred_file,label_file)
        count+=1
    print('[I] The Mean IOU is', iou*1.0/count)
    

In [ ]:
batch_iou('test_output_AOI','/home/lixg/data/road/labels_pixel')
# 10 35 94 125
# label_name = '../labels_pixel/im_10_mask.png'
# # pred_image = 'test_output_AOI/testing-125.png'
# pred_image = 'other_outputs/object/im_10.png'
# compute_image_iou(pred_image,label_name)